In [6]:
# import libraries 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from bs4 import BeautifulSoup
import pprint
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import glob

In [7]:
#form all the URLs to Parse
root_url="https://www.prokabaddi.com/stats/"
player_matrix=['102-total-points-statistics' , 
               '21-successful-raids-statistics' , 
               '22-raid-points-statistics' , 
               '23-successful-tackles-statistics' , 
               '103-tackle-points-statistics' , 
               '138-avg-raid-points-statistics' , 
               '139-avg-tackle-points-statistics' , 
               '132-do-or-die-raid-points-statistics' , 
               '104-super-raids-statistics' , 
               '28-super-tackles-statistics' , 
               '100-super-10s-statistics' , 
               '101-high-5s-statistics'
              ]
team_matrix = [ '134-super-raid-statistics' , 
                '20-super-tackles-statistics' , 
                '136-all-outs-inflicted-statistics' , 
                '137-all-outs-conceded-statistics'
              ]

season_list = [x for x in [[1,2,3,4,8,26,49]]
team_mat_target_urls=[root_url+str(season)+'-'+team_mat for season in season_list for team_mat in team_matrix]
player_mat_target_urls=[root_url+str(season)+'-'+play_mat for season in season_list for play_mat in player_matrix]

In [3]:
record_source = {}
info = pd.DataFrame(columns=('rank', 'name','type', 'matchs_played','score', 'id', 'profile_name', 'team', 'season', 'matrix_code', 'matrix_name' ))
def get_full_source_player(url):
    season_details = url.split('/')[-1].split('-',2)
    driver = webdriver.Chrome(r"C:\Users\sumsatap\Downloads\chromedriver_win32\chromedriver.exe")
    driver.get(url)
    driver.maximize_window()
    driver.implicitly_wait(1)
    while True:
        try:
            driver.find_element_by_xpath('//*[@id="load_more"]/span').click()
            time.sleep(1)
        except:
            page_src = driver.page_source
            soup = BeautifulSoup(page_src, 'lxml')
            div_span = []
            div_a = []
            for div in soup.find_all('div', {"class": "sipk-lb-detailItem wl-team-detail"}):
                div_span.append(div.find_all('span'))
                ous=[]
                for i in range(len(div_span)):
                    ins=[]
                    for elem in div_span[i]:
                        ins.append(elem.text.replace('\n','').strip())
                    info.loc[i,:len(ins)]=ins
                div_a.append(div.find_all('a'))
                for i in range(len(div_a)):
                    ins=[]
                    for a in div_a[i]:
                        ins.append(a.attrs['href'])
                    #print(ins)
                    info.loc[i,5:8]=ins
            info[['season', 'matrix_code', 'matrix_name']] = season_details
            info.to_csv('_'.join(season_details)+'.csv')
            driver.quit()
            break

In [4]:
for url in player_mat_target_urls:
    get_full_source_player(url)

In [8]:
info = pd.DataFrame(columns=('rank', 'team', 'matchs_played','score', 'id', 'profile_name', 'season', 'matrix_code', 'matrix_name' ))
def get_full_source_team(url):
    season_details = url.split('/')[-1].split('-',2)
    driver = webdriver.Chrome(r"C:\Users\sumsatap\Downloads\chromedriver_win32\chromedriver.exe")
    driver.get(url)
    driver.maximize_window()
    driver.implicitly_wait(1)
    while True:
        try:
            driver.find_element_by_xpath('//*[@id="load_more"]/span').click()
            time.sleep(1)
        except:
            page_src = driver.page_source
            soup = BeautifulSoup(page_src, 'lxml')
            div_span = []
            div_a = []
            for div in soup.find_all('div', {"class": "sipk-lb-detailItem wl-team-detail"}):
                div_span.append(div.find_all('span'))
                ous=[]
                for i in range(len(div_span)):
                    ins=[]
                    for elem in div_span[i]:
                        ins.append(elem.text.replace('\n','').strip())
                    #print(ins)
                    info.loc[i,:len(ins)]=ins
                div_a.append(div.find_all('a'))
                for i in range(len(div_a)):
                    ins=[]
                    for a in div_a[i]:
                        ins.append(a.attrs['href'])
                    #print(ins)
                    info.loc[i,4:6]=ins
            info[['season', 'matrix_code', 'matrix_name']] = season_details
            info.to_csv('_'.join(season_details)+'.csv')
            driver.quit()
            break

In [9]:
for url in team_mat_target_urls:
    get_full_source_team(url)

In [10]:
all_matrix={}
for pmat in player_matrix:
    code, matrix=pmat.split('-',1)
    all_matrix['player_'+matrix]=(pd.concat([pd.read_csv(f) for f in glob.glob('*_'+code+'_*.csv')], ignore_index = False).iloc[:,1:])
for tmat in team_matrix:
    code, matrix=tmat.split('-',1)
    all_matrix['team_'+matrix]=(pd.concat([pd.read_csv(f) for f in glob.glob('*_'+code+'_*.csv')], ignore_index = False).iloc[:,1:])    

In [11]:
for table_name in all_matrix.keys():
    curr_df = all_matrix.get(table_name)
    curr_df = curr_df[curr_df.isnull().sum(axis=1) == 0]
    all_matrix[table_name] = curr_df
    
    if table_name.startswith('team'):
        
        split_df = curr_df['profile_name'].str.split("-profile-", n = 1, expand = True)
        curr_df["profile_name"] = split_df[0]
        curr_df["team_id"] = split_df[1]
    if table_name.startswith('player'):
        
        split_df = curr_df['profile_name'].str.split("-profile-", n = 1, expand = True)
        curr_df["profile_name"] = split_df[0]
        curr_df["player_id"] = split_df[1]
        
        split_df = curr_df['team'].str.split("-profile-", n = 1, expand = True)
        curr_df["team_name"] = split_df[0]
        curr_df["team_id"] = split_df[1]
        curr_df = curr_df.drop('team',axis=1)
        
    curr_df = curr_df.drop(['id','matrix_code','matrix_name'],axis=1)
    curr_df.loc[curr_df['season'] == 8, 'season'] = 5
    curr_df.loc[curr_df['season'] == 26, 'season'] = 6
    curr_df.loc[curr_df['season'] == 49, 'season'] = 7
    
    all_matrix[table_name] = curr_df
    curr_df.to_csv(table_name+'_final.csv')